In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary



C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
# Datas preprocessing methods.
# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

def lemmatize(article):
    output = []
    outPonc = article.translate(article.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [4]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [7]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

# To create ours delimiters, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    if not isinstance(data['abstract'][i], float): #if not null, empty, or whatever (so if there is a abstract)
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
        numArti+=1
        usable.append(stemmer.stem(lemmatize(data['abstract'][i])))
limits.append(numArti)



In [8]:
# Display pre-processed datas
print("nombre d'articles =", len(usable))
print("limits =", limits)

usable[0]

nombre d'articles = 1096
limits = [267, 543, 790, 1096]


'plateforme objectif permettre citoyen danalyserpar euxmême tweet politique dévénement spécifique francepour cas lélection présidentiel 2017 idéo2017 analyser quasitemp réel message candidat fournir principal caractéristiqueslusage lexiqu politique comparaison entrer candidat'

In [9]:
#params
nb_concepts = 30
min_gram = 1
max_gram = 3

# Creation of cleandocs, which is usable[] with ngrams
cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

# Creation of tfidf model, a tool to create ours tfidf
corpus = []
dictionary = corpora.Dictionary(cleandocs)
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)

# Creation of partitions_lsa[], which give us the LSA of each partition
partitions_lsa = []
beg = 0
for l in limits:
    last = l
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
    corpus_lsi = lsi[corpus_tfidf[beg:last]]
    partitions_lsa.append(corpus_lsi)
    beg = l

In [11]:
num_partition = 0
for lsa in partitions_lsa:
    print("Partition numéro:",num_partition)
    num_partition+=1
    i=0
    for doc in lsa:
        if (i<3):
            print("document number ", i)
            i+=1
            print(doc)


Partition numéro: 0
document number  0
[(0, 0.005489804287081122), (1, -0.02791722928535486), (2, 0.02128972433959377), (3, 0.021841665064694406), (4, 0.00866254034448971), (5, -0.015127524434327607), (6, -0.009415870565543535), (7, -0.0031278095778846573), (8, -0.0053132244009419256), (9, 0.012568609869497625), (10, 0.021823790495537867), (11, 0.03172812311482545), (12, 0.008646554273059718), (13, 0.0060853445937769), (14, 0.02066749567828601), (15, -0.0062804065234330405), (16, -0.03698914111220524), (17, -0.0008958618367283544), (18, -0.045212595741772894), (19, -0.02626303794496773), (20, 0.028286125043894047), (21, 0.004543855206786278), (22, -0.007202135463788318), (23, 0.02690517534503017), (24, 0.001453461443122326), (25, 0.01672230382157509), (26, -0.02686823685900035), (27, -0.012687905648606007), (28, -0.00830627582391572), (29, -0.004038838092350947)]
document number  1
[(0, 0.0114591714404101), (1, -0.09594275228517507), (2, 0.001832348586869282), (3, -0.07118575251456316)

[(0, 0.00821155489185676), (1, 0.0931840468424834), (2, 0.021496445063474003), (3, 0.0070277465744208656), (4, -0.08830669975940213), (5, -0.05639368396998693), (6, 0.006352822742294887), (7, -0.027995691999516105), (8, 0.0167255694609459), (9, -0.019275608941014308), (10, -0.004150847459637002), (11, -0.003424102652262867), (12, -0.003002531757179119), (13, -0.017849262512527893), (14, 0.021865306569477917), (15, -0.04812813992077054), (16, 0.025100052428343875), (17, 0.015335984955540546), (18, -0.0015579561259903667), (19, 0.021156813874777903), (20, 0.016609335199838048), (21, -0.03373740437020239), (22, -0.02801475048262389), (23, -0.04812077108695218), (24, 0.02269044989211623), (25, 0.02293032246293893), (26, -0.07260096955309343), (27, -0.027525695563188863), (28, 0.018467804408471795), (29, 0.0672554944576633)]
document number  1
[(0, 0.007260481442496487), (1, 0.07291021263132706), (2, 0.010633989807097507), (3, 0.005415712810193173), (4, -0.006375105231577169), (5, -0.012289

In [18]:
# Let's create ours partitions
partitions = []

# You must specify a treshold, to know what are the doc you keep, and what are the doc you drop
tresh = 0.03

for corpus_lsi in partitions_lsa:
    # Let's create ours clusters
    clusters = []

    for i in range(0, nb_concepts):
        dic = {}
        num_doc = 0
        for doc in corpus_lsi:
            if abs(doc[i][1]) > tresh:
                dic[num_doc] = doc[i][1]
            num_doc+=1
        clusters.append(dic)
    partitions.append(clusters)
    
# TODO: it would be nice to know how many articles are in no cluster anymore

In [19]:
# Display clusters 3 of partition 0 
partitions[0][3]

{1: -0.07118575251456316,
 2: 0.03517684316237265,
 4: 0.030949588077932168,
 6: -0.03903031658883216,
 10: -0.042122287867876196,
 13: 0.03199465425799143,
 15: 0.05696508686371263,
 19: 0.03070633463928613,
 21: 0.044167731599077685,
 27: 0.07214586367724328,
 30: 0.0489790056285975,
 35: 0.03491735862798161,
 39: -0.04648167279744784,
 44: 0.06719455009309083,
 45: -0.03236240075430082,
 50: -0.07234008899732827,
 55: 0.046952221602755286,
 56: 0.07284638968514465,
 59: -0.0426280183106404,
 60: -0.10906514237008724,
 69: -0.09380809963696331,
 76: 0.03664858548972769,
 78: 0.05091243581423806,
 82: 0.04197132864238727,
 83: 0.03764239541212245,
 86: -0.04441379782253787,
 88: 0.030454056151978385,
 93: -0.0364870257285272,
 97: 0.06521992853113161,
 99: -0.10578249336226304,
 100: -0.07286352803549878,
 101: 0.03651901036362523,
 105: 0.057555840428314284,
 111: -0.03295084517195956,
 112: 0.06425700594567908,
 113: 0.046872105666898,
 117: 0.0426939060544083,
 119: -0.110881429266

In [ ]:
nb_labels_by_cluster = 5

# Let's labelize our clusters
# For this, we will use the tfidf matrix

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# We can access the value in the tfidf using:
#tfidf.toarray().item(num_doc, num_word)
# To know the number of the word searched, we will use:
#vectorizer.vocabulary_[word]

# take less than 8h to compute x)
labels = []
for clusters in partitions:
    l = []
    for clus in clusters:
        first_arti = True
        for article in clus:
            link = abs(clus[article])
            if first_arti:
                coef_list = (tfidf.toarray()[article] * link)
                first = False
            else:
                # the more an article have a high coeficient, the more he is implied in the labeling step
                coef_list += (tfidf.toarray()[article] * link)
        # Now we have coef_list filled by every coeficient in the multiple tfidf
        # Let's find the best ones, to finally get the labels
        res = dict(zip(vectorizer.get_feature_names(), coef_list))

        l.append(Counter(res).most_common(nb_labels_by_cluster))
    labels.append(l)

# TODO: on observe beaucoup de labels identiques entre deux clusters
# Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [ ]:
# Display labels
# labels is composed by an array for each partition
labels